# 이미지 사이즈 옵션
- (yolo - optional) 416 -> (efficientnet) 224 (preferred)

# 구글 드라이브 연동 및 import

In [6]:
# 구글 드라이브 연동하기
from google.colab import drive
drive.mount('/content/drive')


# 경로 및 파일 관리
import os
import shutil

# yolo5 파일에서 필요한 것을 설치하기 위한 경로 설정하기
os.chdir('/content/drive/MyDrive/yolo_efficientnet/yolov5')

# 환경 설정 파일 설치 -> [변경 내용] requests==2.31.0   pillow<10.1.0
%pip install -qr requirements.txt

# 공유 데이터셋 파일을 가져오기 위한 roboflow
%pip install -q roboflow

# 딥러닝 라이브러리
import torch

# yaml : json 또는 xml 처럼 데이터 송수신을 위해 사전에 정해진 규칙을 따르는 '데이터 전송 파일' 형식
import yaml

# 이미지 보여주는 용도
from PIL import Image as Img
from IPython.display import Image, display # clear_output

# 특정 확장자 파일 불러오기 -> glob.glob() : 특정 패턴과 일치하는 모든 경로명을 리스트로 반환
import glob

# 학습한 베스트 모델 다운로드하기
from google.colab import files



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터셋 정의 및 경로 설정하기

In [7]:
# 데이터셋 파일 이름 정의 -> 해당 데이터셋의 이미지 =>  [연습] : 이미지 사이즈는 그대로 두고 학습코드에서 --imgsz로 resize 하기, [실전] : 416으로 리사이즈한 것으로 이미지 구축

dataset_file_name = 'my_custom_datasets'

# 사용할 데이터셋 경로 지정 하기
dataset_directory = f'/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/{dataset_file_name}'

# 데이터셋의 yaml 파일 경로 지정하기
dataset_yaml_directory = f'/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/{dataset_file_name}/data.yaml'

# yaml 파일 불러오기
with open(dataset_yaml_directory) as file :
    dataset_yaml = yaml.load(file, Loader = yaml.FullLoader)
    display(dataset_yaml)

# yaml 파일 train, val 경로 수정하기
dataset_yaml['train'] = f'/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/{dataset_file_name}/train/images' # train.py로 학습하기
dataset_yaml['val'] = f'/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/{dataset_file_name}/valid/images' # val.py로 검증하기
dataset_yaml['test'] = f'/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/{dataset_file_name}/test/images' # detect.py로 검증하기

# 수정한 경로 반영
with open(dataset_yaml_directory, 'w') as file:
    yaml.dump(dataset_yaml, file)

# 경로 반영 확인하기
print('변경된 yaml 파일 : ')
with open(dataset_yaml_directory) as file:
    dataset_yaml = yaml.load(file, Loader=yaml.FullLoader)
    display(dataset_yaml)

{'names': ['top', 'bottom', 'outer', 'skirt', 'dress'],
 'nc': 5,
 'test': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/test/images',
 'train': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/train/images',
 'val': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/valid/images'}

변경된 yaml 파일 : 


{'names': ['top', 'bottom', 'outer', 'skirt', 'dress'],
 'nc': 5,
 'test': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/test/images',
 'train': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/train/images',
 'val': '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/valid/images'}

# 전이학습 : 사전학습 파라미터를 이용해서 학습하기

In [ ]:
# 사전학습 파라미터 정의
pretrained_best_pt_path =  f'/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best.pt'

# 학습 코드
!python train.py --imgsz 416 --batch 64 --epochs 100 --data {dataset_yaml_directory} --weights {pretrained_best_pt_path}


# 파라미터 저장 위치 : Results saved to runs/train/exp4

2024-07-03 06:03:52.564107: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 06:03:52.564159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 06:03:52.565691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best.pt, cfg=, data=/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None

In [ ]:
# 텐서보드로 학습 방향성 파악하기

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 저장된 최종 best.pt 경로 설정하기
train_exp_num = 4

trained_best_pt_path = f'/content/drive/MyDrive/yolo_efficientnet/yolov5/runs/train/exp{train_exp_num}/weights/best.pt'

In [ ]:
# 검증 결과 확인 후 최종 best.pt 따로 저장하기

# 저장 경로 설정
best_pt_save_root_path = f'/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/'

# 저장 이름 설정
save_name = f'final_fashion_yolo_best_2.pt'

# 복사 후 저장
shutil.copy(trained_best_pt_path, best_pt_save_root_path + save_name)

'/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best_2.pt'

### 검증하기

In [8]:
# 검증 및 테스트 경로 설정
test_image_path = '/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/test/images'

train_pt_path = '/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best2.pt'

# 검증 및 테스트하기
!python detect.py --conf-thres 0.3 --weights {trained_best_pt_path}  --data {dataset_yaml_directory} --imgsz 416 --source {test_image_path}

# 검증 결과 저장 경로 : runs/detect/exp


detect: weights=['/content/drive/MyDrive/yolo_efficientnet/yolov5/runs/train/exp4/weights/best.pt'], source=/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/test/images, data=/content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/data.yaml, imgsz=[416, 416], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/582 /content/drive/MyDrive/yolo_efficientnet/yolo_dataset/my_custom_datasets/test/images/바지_데님ᄑ

In [ ]:
# 검증 결과 확인하기

'/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best.pt'

- 테스트 이미지  yolo 추출 결과 확인

In [ ]:
!pip install extcolors

In [ ]:
# 딥러닝 사용
import torch

import extcolors

# 특정 확장자 파일 불러오기 -> glob.glob() : 특정 패턴과 일치하는 모든 경로명을 리스트로 반환
import glob

# 이미지 관련
from PIL import Image as Img
from PIL import ImageFile
import cv2
import numpy as np
import glob
from IPython.display import display

# 파일 관리
import os

# efficientNet_v2_s 모델을 가지고오기 위함
import torchvision.models as get_model
import torchvision

# input 데이터를 정규화하기 위한 transform 규칙을 적용하기 위함
import torchvision.transforms as transforms

# optimizer를 가지고 오기 위함.
import torch.optim as optim

# pytorch에서 지원하는 다양한 계층 및 손실함수 계산을 사용하기 위함.
import torch.nn as nn

from torchvision.datasets import ImageFolder
from torchvision import transforms # ToTensor, Resize, Compose
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

# 스케쥴러
from torch.optim.lr_scheduler import OneCycleLR

In [ ]:
color_ranges = {
    "red": ((128, 0, 0), (255, 127, 127)),
    "green": ((0, 128, 0), (127, 255, 127)),
    "blue": ((0, 0, 128), (127, 127, 255)),
    "yellow": ((128, 128, 0), (255, 255, 127)),
    "magenta": ((128, 0, 128), (255, 127, 255)),
    "cyan": ((0, 128, 128), (127, 255, 255)),
    "orange": ((128, 64, 0), (255, 191, 127)),
    "purple": ((64, 0, 64), (191, 127, 191)),
    "pink": ((255, 0, 127), (255, 191, 255)),
    "lime": ((0, 255, 0), (191, 255, 191)),
    "brown": ((64, 32, 0), (191, 127, 64)),
    "gray": ((128, 128, 128), (191, 191, 191)),
    "black": ((0, 0, 0), (63, 63, 63)),
    "white": ((192, 192, 192), (255, 255, 255))
}


def get_color_name(rgb_value):
    for color_name, (lower_bound, upper_bound) in color_ranges.items():
        # 특정 색깔 범위에 있으면
        if all(lower_bound[i] <= rgb_value[i] <= upper_bound[i] for i in range(3)):
            return color_name
    return "UNKNOWN"

In [ ]:
def maintain_proportion_and_resize_by_cv2(image_path, size=(416, 416)):

    # 이미지 읽기(BGR)
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 열 수 없습니다: {image_path}")

    # 이미지 형식 변경하기(BGR -> RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 높이, 너비 추출
    original_height, original_width = image.shape[:2]

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # 새로운 이미지를 패딩하여 416x416 크기로 만들기
    new_image = np.full((size[1], size[0], 3), (255, 255, 255), dtype=np.uint8)  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return new_image


def maintain_proportion_and_resize_by_pil(image, size=(416, 416)):

    # PIL.Image 형식의 이미지를 비율을 유지하면서 지정된 크기로 리사이즈하고,
    # 흰색 배경으로 패딩하여 최종 이미지를 반환

    # 원본 이미지 크기 추출
    original_width, original_height = image.size

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = image.resize((new_width, new_height), Img.ANTIALIAS)

    # 새로운 이미지를 패딩하여 지정된 크기로 만들기
    new_image = Img.new("RGB", size, (255, 255, 255))  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image.paste(resized_image, (x_offset, y_offset))

    return new_image

In [ ]:
# device 설정
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

# yolo pt 파일 저장 경로 정의
yolov5_best_pt_save_path = '/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best.pt'

# 학습 가중치를 적용한 모델 불러오기
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path= yolov5_best_pt_save_path )

# device 설정
yolov5_model.to(device)

# yolov5_model 평가 모드 선언
yolov5_model.eval()

In [ ]:
# test 이미지 정의
test_image_path = '/content/drive/MyDrive/yolo_efficientnet/테스트4.jpg'

# natural_image 만들기
natural_img = cv2.imread(test_image_path)
natural_img = cv2.cvtColor(natural_img, cv2.COLOR_BGR2RGB)
natural_img = Img.fromarray(natural_img)

# [원본 이미지✅]
print('원본 이미지')
display(natural_img)


temp = [224, 360, 412, 512, 625, 730]

for i, tmp in enumerate(temp) :

    # resize 값 설정
    value_of_resize = (tmp, tmp)

    resized_img = maintain_proportion_and_resize_by_cv2(test_image_path, value_of_resize)
    resized_img_for_cropping = maintain_proportion_and_resize_by_cv2(test_image_path, value_of_resize)

    result = yolov5_model(resized_img)

    if len(result.xyxy[0]) != 0:
        print(f'{len(result.xyxy[0])}개의 탐지 결과가 있습니다')

    else :
        print('탐지 결과가 없습니다.')

    detected_image_composition_list = []

    for i, bounding_box in enumerate(result.xyxy[0]):
        xmin, ymin, xmax, ymax = map(int, bounding_box[:4])
        cls = int(bounding_box[5])
        # [대분류 정보✅]
        class_name = result.names[cls]
        confidence = round(bounding_box[4].item(), 2)

        # bounding box 보여주기
        cropped_img = resized_img_for_cropping[ymin:ymax, xmin:xmax]
        cropped_img = Img.fromarray(cropped_img, 'RGB')
        detected_image_composition_list.append(cropped_img)
        # [bounding box(들)✅]
        display(cropped_img)


        colors, pixel_count = extcolors.extract_from_image(cropped_img)
        # [색상 정보 추출✅]
        print(f'{test_image_path.split("/")[-1]}에서 탐지한 {i+1}번쨰 옷의 대분류는 {class_name}입니다.  가장 많이 등장한 색상은 {get_color_name(colors[0][0])} 입니다. -> 결과 신뢰도 : {confidence * 100}')

In [ ]:
image = cv2.imread(test_image_path)

image.shape

(1280, 720, 3)